# Generador de personas aleatorias

In [1]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import pytz
import re
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
from typing import Union

In [16]:
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac.replace(' ', '_').lower())
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento segun RENAPO (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS', 'baja_california': 'BC', 'baja_california_sur': 'BS',
		'campeche': 'CC', 'coahuila': 'CL', 'colima': 'CM', 'chiapas': 'CS', 'chihuahua': 'CH',
		'distrito_federal': 'DF', 'ciudad_de_mexico': 'DF', 'durango': 'DG', 'guanajuato': 'GT',
		'guerrero': 'GR', 'hidalgo': 'HG', 'jalisco': 'JC', 'mexico': 'MC', 'estado_de_mexico': 'MC',
		'michoacan': 'MN', 'morelos': 'MS', 'nayarit': 'NT', 'nuevo_leon': 'NL', 'oaxaca': 'OC',
		'puebla': 'PL', 'queretaro': 'QT', 'quintana_roo': 'QR', 'san_luis_potosi': 'SP', 'sinaloa': 'SL',
		'sonora': 'SR', 'tabasco': 'TC', 'tamaulipas': 'TS', 'tlaxcala': 'TL', 'veracruz': 'VZ',
		'yucatan': 'YN', 'zacatecas': 'ZS'
	}
	if estado_nac in clave_estados:
		curp += clave_estados[estado_nac]
	else: curp += 'NE'
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

def obtener_coordenadas(ciudad, estado, pais = 'Mexico') -> dict:
	try:
		# Hacer peticion a un api para obtener coordenadas del centro y extremas
		geolocator = Nominatim(user_agent = 'geoapiExercises')
		location = geolocator.geocode(f'{ciudad} {estado} {pais}')
		# En caso de que no se haya encontrado nada
		if not location: raise BaseException(f'No se encontraron coordenadas de "{ciudad}", "{estado}", "{pais}"')
		location = location.raw
		# Obtener coordenadas del centro
		centro = (float(location['lat']), float(location['lon']))
		# Obtener extremos originales
		minx, maxx, miny, maxy = [float(i) for i in location['boundingbox']]
		ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
		print(f'\033[33mobtener_coordenadas()\033[0m: Coordenadas encontradas en "{ciudad}", "{estado}", "{pais}"')
	except BaseException as e:
		print(f'\033[31mobtener_coordenadas()\033[0m:', e)
		return {}
	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

def generar_puntos_random(coords: dict, n = 1, reducir = True) -> list:
	coords_necesarias = ('centro', 'suroeste', 'noreste')
	if not all(k in coords for k in coords_necesarias):
		print('\033[33mgenerar_puntos_random()\033[0m: No se encontro alguna de las coordenadas necesarias:', coords_necesarias)
		return []
	# Obtener coordenadas del centro
	cenx, ceny = coords['centro']
	# Obtener extremos originales
	minx, miny = coords['suroeste']
	maxx, maxy = coords['noreste']
	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

def buscar_direccion(puntos_random: list) -> dict:
	geolocator = Nominatim(user_agent = 'geoapiExercises')
	dirrr = {}
	listo = False
	for punto_random in puntos_random:
		dirr = {}
		address = geolocator.reverse(punto_random, language = 'es_MX')
		if not address: continue
		direccion = address.raw['address']
		print(direccion)
		# Ver que campos se encontraron
		# https://nominatim.org/release-docs/latest/api/Output/#addressdetails
		# amenity, building, shop
		if any(k in direccion for k in ('amenity', 'building', 'shop', 'office', 'leisure')):
			if 'amenity' in direccion: dirr['local'] = direccion['amenity']
			elif 'building' in direccion: dirr['local'] = direccion['building']
			elif 'shop' in direccion: dirr['local'] = direccion['shop']
			elif 'office' in direccion: dirr['local'] = direccion['office']
			elif 'leisure' in direccion: dirr['local'] = direccion['leisure']
		# road
		if 'road' in direccion: dirr['calle'] = direccion['road']
		# house_number
		if 'house_number' in direccion: dirr['numero'] = direccion['house_number']
		else: dirr['numero'] = 'S/N'
		# neighbourhood, residential, suburb, quarter
		if any(k in direccion for k in ('neighbourhood', 'residential', 'suburb', 'quarter')):
			if 'neighbourhood' in direccion: dirr['colonia'] = direccion['neighbourhood']
			elif 'residential' in direccion: dirr['colonia'] = direccion['residential']
			elif 'suburb' in direccion: dirr['colonia'] = direccion['suburb']
			elif 'quarter' in direccion: dirr['colonia'] = direccion['quarter']
		# postcode
		if 'postcode' in direccion: dirr['cp'] = direccion['postcode']
		# city, town, village, hamlet
		if any(k in direccion for k in ('city', 'town', 'village', 'hamlet')):
			if 'city' in direccion: dirr['ciudad'] = direccion['city']
			elif 'town' in direccion: dirr['ciudad'] = direccion['town']
			elif 'village' in direccion: dirr['ciudad'] = direccion['village']
			elif 'hamlet' in direccion: dirr['ciudad'] = direccion['hamlet']
		# county
		if 'county' in direccion: dirr['municipio'] = direccion['county']
		# state
		if 'state' in direccion: dirr['estado'] = direccion['state']
		# country
		if 'country' in direccion: dirr['pais'] = direccion['country']
		# Si se encontro al menos calle y cp
		if all(k in dirr for k in ('calle', 'numero', 'cp')): listo = True
		# Si ya se encontro una buena direccion, dejar de buscar
		if listo:
			print('\033[33mbuscar_direccion()\033[0m: Direccion encontrada en', punto_random)
			print('\033[32m', dirr, '\033[m')
			dirrr = dirr
			break
	if not listo: print('\033[33mbuscar_direccion()\033[0m: No se encontro una direccion precisa')
	return dirrr

In [15]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv', header = None)
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for i in range(n):
		print('Idx', i)
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		if edad <= reap['edad'].max(): criter = (reap['edad'] == edad)
		else: criter = (reap['edad'] == reap['edad'].max())
		margen_ap = reap[criter].to_dict(orient = 'records')[0]

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'], margen_ap['lim_sup.altura.h.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'], margen_ap['lim_sup.peso.h.kg']), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'], margen_ap['lim_sup.altura.m.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'], margen_ap['lim_sup.peso.m.kg']), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()
		ct_ciudad_index_row = ct_ciudad.index.values[0]
		ct_ciudad = ct_ciudad.to_dict(orient = 'records')[0]

		ciudad = ct_ciudad['ciudad']
		estado = ct_ciudad['estado']
		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)
		tel = f'({ct_ciudad["lada"]})' + ' ' + '{:03d}'.format(rm.randint(0, 999)) + ' ' + '{:04d}'.format(rm.randint(0, 9999))
		profesion = profesiones.sample().values[0][0] if edad >= 18 else None

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		nss = None
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			# suponiendo que la persona empezo a trabajar de sus 18 a sus 25
			nss2 = (fecha_nac + pd.DateOffset(months = rm.randint(216, 300))).strftime('%y')
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%04d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'

		"""Coordenadas de la ciudad"""
		centro = coordenadas = None
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['centro']) and pd.notna(ct_ciudad['suroeste']) and pd.notna(ct_ciudad['noreste']):
			centro = eval(ct_ciudad['centro'])
			coordenadas = {
				'centro': centro,
				'suroeste': eval(ct_ciudad['suroeste']),
				'noreste': eval(ct_ciudad['noreste'])
			}
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			try:
				coordenadas = obtener_coordenadas(ciudad, estado)
				if coordenadas:
					centro = coordenadas['centro']
					# guardar coordenadas en el catalogo
					# para que a la otra no haya que buscar en internet
					ct_ciudades.loc[ct_ciudad_index_row, 'centro'] = str(centro)
					ct_ciudades.loc[ct_ciudad_index_row, 'suroeste'] = str(coordenadas['suroeste'])
					ct_ciudades.loc[ct_ciudad_index_row, 'noreste'] = str(coordenadas['noreste'])
					ct_ciudades.to_csv(file_ciudades, index = False)
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print(f'\033[31mCoordsException: idx {i}\033[0m')
				print(be)

		"""Direccion"""
		direcc = cp = None
		if centro and coordenadas:
			puntos_random = generar_puntos_random(coordenadas, 5)
			try:
				direccion = buscar_direccion(puntos_random)
				if direccion and all(k in direccion for k in ('calle', 'numero', 'cp')):
					cp = direccion['cp']
					direcc = f"{direccion['calle']}, {direccion['numero']}"
			except BaseException as be:
				print(f'\033[31mAddressException: idx {i}\033[0m')
				print(be)

		"""Zona horaria"""
		timezone = tz_offset = None
		if centro:
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			#if centro[1] > 180.0 or centro[1] < -180.0 or centro[0] > 90.0 or centro[0] < -90.0:
			timezone = obj.timezone_at(lat = centro[0], lng = centro[1])
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')

		if not centro: centro = (np.nan, np.nan)

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'curp': curp,
			'correo': correo,
			'tel': tel,
			'color_fav': fake.safe_color_name(),
			'profesion': profesion,
			'nss': nss,
			'direccion': direcc,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': centro[0],
			'lon': centro[1],
			'zona_horaria': timezone,
			'tz_offset': tz_offset,
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

Idx 0
{'road': 'Calle Miguel Hidalgo', 'village': 'Tilzapotla', 'county': 'Puente de Ixtla', 'state': 'Morelos', 'country': 'México', 'country_code': 'mx'}
{'road': 'Calle Principal', 'village': 'Tilzapotla', 'county': 'Puente de Ixtla', 'state': 'Morelos', 'country': 'México', 'country_code': 'mx'}
{'road': 'Calle Miguel Hidalgo', 'village': 'Tilzapotla', 'county': 'Puente de Ixtla', 'state': 'Morelos', 'country': 'México', 'country_code': 'mx'}
{'road': 'Calle Acueducto', 'village': 'Tilzapotla', 'county': 'Puente de Ixtla', 'state': 'Morelos', 'country': 'México', 'country_code': 'mx'}
{'road': 'Calle Insurgentes', 'village': 'Tilzapotla', 'county': 'Puente de Ixtla', 'state': 'Morelos', 'country': 'México', 'country_code': 'mx'}
buscar_direccion(): No se encontro una direccion precisa
Idx 1
{'leisure': 'Puerto Aventuras Golf Course', 'road': 'Bahía Xcacel', 'town': 'Puerto Aventuras', 'municipality': 'Solidaridad', 'state': 'Quintana Roo', 'postcode': '77782', 'country': 'México', 

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,curp,...,profesion,nss,direccion,cp,ciudad,estado,lat,lon,zona_horaria,tz_offset
0,None,Eduardo,Marroquín,Ureña,1945-03-03,H,175.78,85.58,B+,MAUE450303HMSARD82,...,Científico de datos,93 66 45 2173 8,None,None,Tilzapotla,Morelos,18.488698,-99.274999,America/Mexico_City,-0500
1,None,Silvia,Hernádez,Madrigal,2017-09-10,M,100.20,15.32,A+,HEMS170910MQREAIX2,...,None,None,"Bahía Xcacel, S/N",77782,Puerto Aventuras,Quintana Roo,20.506668,-87.230063,America/Cancun,-0500
2,Lic.,Adalberto,Villalpando,Oquendo,1908-11-09,H,171.35,67.84,A+,VIOA081109HASIQD95,...,Tecnólogo Arquitectónico,38 32 08 5048 2,"Avenida Convencion de 1914 Oriente, S/N",20250,Aguascalientes,Aguascalientes,21.880487,-102.296719,America/Mexico_City,-0500
3,None,Felix,Leyva,Alvarado,1994-07-28,H,174.17,88.05,B+,LEAF940728HGRELE74,...,Agrimensor comercial/residencial,91 16 94 0354 4,None,None,San Miguel Tecuixiapan,Guerrero,NaN,NaN,None,None
4,None,Gael,Montemayor,Treviño,2008-06-30,H,150.19,34.32,AB+,MOTG080630HDFORAR2,...,None,None,"Calle Cafetal, 565",08400,Iztacalco,Ciudad de Mexico,19.396254,-99.096587,America/Mexico_City,-0500
5,Sra.,Eva,Cazares,Granados,1915-08-15,M,159.73,80.24,A+,CAGE150815MMNARV47,...,Redactor de publicidad,99 38 15 0658 1,None,None,Gabriel Zamora (Lombardia),Michoacan,19.157562,-102.058639,America/Mexico_City,-0500
6,None,Elvira,Jaramillo,Muñoz,1967-04-02,M,156.17,61.93,O+,JAME670402MZSAUL98,...,Oficial de entrenamiento y educación de las fu...,38 87 67 1698 9,None,None,Malpaso,Zacatecas,22.623966,-102.762434,America/Mexico_City,-0500
7,None,Cristal,Niño,Rivas,1976-08-15,M,155.91,70.71,B-,NIRC760815MPLIIR76,...,Administrador de organizaciones benéficas,73 98 76 5356 8,"Calle Libertad, S/N",75610,Tochtepec,Puebla,18.818281,-97.822671,America/Mexico_City,-0500
8,None,Carla,Godínez,Tirado,1934-08-30,M,153.34,68.38,A+,GOTC340830MDFOIA54,...,Tecnólogo de color,86 54 34 8226 8,"Calle Benito Juárez, S/N",14406,Tlalpan,Ciudad de Mexico,19.288420,-99.167051,America/Mexico_City,-0500
9,None,Virginia,Chapa,Pantoja,1982-09-29,M,156.98,71.79,A+,CHPV820929MGRHAI93,...,Astrónomo,02 04 82 5008 1,"Calle 5 de Mayo, S/N",40180,Zumpango del Río,Guerrero,17.650399,-99.526360,America/Mexico_City,-0500
